In [17]:
import requests  # requests 모듈 임포트
import numpy as np
import pandas as pd

headers = {  # 헤더 설정
    'Content-Type': 'application/json'  # JSON 형식 설정
}

#https://apihub.kma.go.kr/ 참조, 기상청 api 페이지, url 형식에 따라 요청되는 데이터가 다름
#php?다음부분에 요청 변수명, 값적고 & 붙이기, 마지막은 authkey로 api키 값이 들어가야함
#예시 : reg=11B20702 -> 광주, tmfc=0 시간설정으로 0이면 전체 시간 조회의미
'''
즉, 아래 url의 의미는 단기 예보 구역이 뭐있는지 조회할건데 그중 광주의 전체 예보 시간 조회해줘라는 의미인데, 이 url만 작동제대로 안함, 그냥 전체 데이터 불러와서
확인하는 용도로 사용하면 될듯하며 설명을 위해 이렇게 수정해보았음
'''
url = 'https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?reg=11B20702&tmfc=0&authKey=wgUybypBT-2FMm8qQT_tbQ'  # API URL 설정
response = requests.get(url, headers=headers)  # GET 요청

print(response.text)  # 응답 출력

#START7777
#--------------------------------------------------------------------------------------------------
#  단기예보구역 조회
#---------------------------------------------------------------------------------------------------
#  1. REG_ID   : 예보구역코드
#  2. TM_ST    : 시작시각(년월일시분,KST)
#  3. TM_ED    : 종료시각(년월일시분,KST)
#  4. REG_SP   : 특성 (A:육상광역,B:육상국지,C:도시,D:산악,E:고속도로,H:해상광역,I:해상국지,J:연안바다,K:해수욕장,L:연안항로,M:먼항로,P:산악)
#  5. REG_NAME : 예보구역명
#---------------------------------------------------------------------------------------------------
# REG_ID TM_ST        TM_ED        REG_SP REG_NAME
11000000 199001010000 210012310000 A      육상 
11A00000 201501221100 210012310000 A      서해5도 
11A00100 199001010000 210012310000 B      서해5도 
11A00101 201610131800 210012310000 C      백령도 
11A00102 199001010000 210012310000 C      연평도 
11A00103 199001010000 210012310000 C      소청도 
11B00000 201101110800 210012310000 A      서울.인천.경기 
11B000D1 200707250000 210012310000 D      북한.도봉산 
11B000D2 200204240000 2100

In [79]:
#단기 육상 예보 정보 조회 예시

url = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=11F20501&tmfc1=2025011304&tmfc2=2025011320&disp=0&help=1&authKey=wgUybypBT-2FMm8qQT_tbQ'

response = requests.get(url, headers=headers)  # GET 요청

print(response.text)  # 응답 출력, 텍스트 파일을 전송해줌, json, xml 형식 x, array의 22번 = 컬럼정보, 23번부터 데이터 불러옴

#START7777
#--------------------------------------------------------------------------------------------------
#  단기예보 육상 조회 [입력인수형태][예] ?reg=&tmfc1=2013121018&tmfc2=2013121106&disp=0&help=1
#--------------------------------------------------------------------------------------------------
#  1. REG_ID   : 예보구역코드
#  2. TM_FC    : 발표시각(년월일시분,KST)
#  3. TM_EF    : 발효시각(년월일시분,KST)
#  4. MOD      : 구간 (A01(24시간),A02(12시간))
#  5. NE       : 발효번호
#  6. STN      : 발표관서
#  7. C        : 발표코드
#  8. MAN_ID   : 예보관ID
#  9. MAN_FC   : 예보관명
# 10. W1       : 풍향1(16방위)
# 11. T        : 풍향경향(1:-, 2:후)
# 12. W2       : 풍향2(16방위)
# 13. TA       : 기온
# 14. ST       : 강수확률(%)
# 15. SKY      : 하늘상태코드 (DB01(맑음),DB02(구름조금),DB03(구름많음),DB04(흐림))
# 16. PREP     : 강수유무코드 (0(없음),1(비),2(비/눈),3(눈),4(눈/비(~'19.6.4.),소나기('19.6.4~)))
# 17. WF       : 예보
#--------------------------------------------------------------------------------------------------
# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC 

In [71]:
#텍스트 데이터 처리예시
text_example = response.text.split('\n')[23] # 첫째줄 데이터만 불러옴, \n개행문자를 기준으로 스플릿해서 데이터를 나눔
text_example = np.array(text_example.split(' ')) # 데이터 구분을 공백을 이용해서 해놓았음, 그래서 split을 공백기준으로 실행해서 나눠줌, 연산을 위해 numpy선언
text_example = text_example[text_example != ''] #공백을 여러개 사용한 부분도 있어서 공백만 저장된 부분을 제거후 출력
print(text_example) 
for i, n in enumerate(text_example):
    print(i,n)

['11B20702' '202403111100' '202403111200' 'A02' '0' '109' '2' 'tjwjdrnr'
 '서정국' 'SE' '1' 'S' '10' '60' 'DB04' '1' '"흐리고' '한때' '비"']
0 11B20702
1 202403111100
2 202403111200
3 A02
4 0
5 109
6 2
7 tjwjdrnr
8 서정국
9 SE
10 1
11 S
12 10
13 60
14 DB04
15 1
16 "흐리고
17 한때
18 비"


In [72]:
#판다스 데이터 프레임에 저장, 예보구역 코드, 시간과 기상정보, 예보정보만을 일단 저장함
column = np.array(['REG_ID','TM_FC','TM_EF','W1','T','W2','TA','ST','SKY','PREP','WF'])
text_example = np.concatenate( (text_example[:3], text_example[9:16], np.array([' '.join(text_example[16:]).strip('"')])) , axis = 0)

In [74]:
text_dataFrame = pd.DataFrame(data = np.array([text_example]), columns = column)
print(text_dataFrame)

     REG_ID         TM_FC         TM_EF  W1  T W2  TA  ST   SKY PREP        WF
0  11B20702  202403111100  202403111200  SE  1  S  10  60  DB04    1  흐리고 한때 비


In [ ]:
'''
#  1. REG_ID   : 예보구역코드
#  2. TM_FC    : 발표시각(년월일시분,KST)
#  3. TM_EF    : 발효시각(년월일시분,KST)
#  4. MOD      : 구간 (A01(24시간),A02(12시간))
#  5. NE       : 발효번호
#  6. STN      : 발표관서
#  7. C        : 발표코드
#  8. MAN_ID   : 예보관ID
#  9. MAN_FC   : 예보관명
# 10. W1       : 풍향1(16방위)
# 11. T        : 풍향경향(1:-, 2:후)
# 12. W2       : 풍향2(16방위)
# 13. TA       : 기온
# 14. ST       : 강수확률(%)
# 15. SKY      : 하늘상태코드 (DB01(맑음),DB02(구름조금),DB03(구름많음),DB04(흐림))
# 16. PREP     : 강수유무코드 (0(없음),1(비),2(비/눈),3(눈),4(눈/비(~'19.6.4.),소나기('19.6.4~)))
# 17. WF       : 예보
'''